In [ ]:
# Add list of linkedin post urls
urls = []
debug = False
watermark_flag = False
watermark_text = None

below function takes the URL and converts url posts to a pdf

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import os, io, base64
from PIL import Image, ImageOps
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from PyPDF2 import PdfMerger


# Create a directory to store cached images
if not os.path.exists('cache'):
    os.makedirs('cache')

for i, url in enumerate(urls, start=1):
    print("Processing post#", i, "url:", url)
    output_pdf = f"output_{i}.pdf"
    options = webdriver.ChromeOptions()
    if not debug:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1920, 1080)

    driver.get(url)

    post_text = driver.find_element(By.XPATH, '//h1/following-sibling::article/div/p[@data-test-id="main-feed-activity-card__commentary"]')
    driver.execute_script("arguments[0].scrollIntoView();", post_text)

    text_screenshot = post_text.screenshot_as_base64

    text_image = Image.open(io.BytesIO(base64.b64decode(text_screenshot)))

    dpi = 1200
    padding = 5
    border = 5
    text_image_with_padding = ImageOps.expand(text_image, padding, (255, 255, 255))
    text_image_with_padding = ImageOps.expand(text_image_with_padding, border, (0, 0, 0))
    text_image_with_padding = ImageOps.expand(text_image_with_padding, padding, (255, 255, 255))

    text_image_with_padding = text_image_with_padding.resize((text_image_with_padding.width * 2, text_image_with_padding.height * 2), Image.LANCZOS)
    image_with_high_resolution = text_image_with_padding.convert('RGB')
    text_image_with_padding.save("text.png", dpi=(dpi, dpi))
    image1_path = 'text.png'
    image1 = text_image_with_padding.convert('RGBA')
    image1_path = 'text.png'

    # Initialize image2_path and image2
    image2_path = ''
    image2 = None

    # Check if the image file already exists in the cache
    if os.path.exists(f'cache/image2_{i}.png'):
        image2_path = f'cache/image2_{i}.png'
        image2 = Image.open(image2_path)
    else:
        # Check if the image element exists before attempting to capture it
        if driver.find_elements(By.XPATH, '//h1/following-sibling::article/div/p[@data-test-id="main-feed-activity-card__commentary"]/parent::div//following-sibling::ul'):
            post_image = driver.find_element(By.XPATH, '//h1/following-sibling::article/div/p[@data-test-id="main-feed-activity-card__commentary"]/parent::div//following-sibling::ul')
            # Scroll to the image element
            driver.execute_script("arguments[0].scrollIntoView();", post_image)
            image_screenshot = post_image.screenshot_as_base64  # Capture as base64
            image = Image.open(io.BytesIO(base64.b64decode(image_screenshot)))
            image_with_high_resolution = image.convert('RGB')
            image_with_high_resolution = ImageOps.expand(image_with_high_resolution, padding, (255, 255, 255))
            image_with_high_resolution = image_with_high_resolution.resize((image_with_high_resolution.width * 2, image_with_high_resolution.height * 2), Image.LANCZOS)
            image_with_high_resolution.save(f"cache/image2_{i}.png", dpi=(dpi, dpi))
            image2_path = f'cache/image2_{i}.png'
            image2 = image_with_high_resolution.convert('RGBA')

    if not debug:
        driver.quit()
    print("image1_path:", image1_path, "image2_path:", image2_path)

    # Calculate the width and height based on the presence of image2
    if image2_path:
        width = max(image1.width, image2.width)
        height = image1.height + image2.height
    else:
        width = image1.width
        height = image1.height

    c = canvas.Canvas(output_pdf, pagesize=(width, height))

    c.setFont("Helvetica", 48)
    c.setFillColorRGB(0.95, 0.95, 0.9)  # Gray color for watermark
    c.saveState()
    c.drawImage(image1_path, 0, height - image1.height, width=image1.width, height=image1.height)
    if(watermark_flag):
        c.rotate(35)  # Rotate watermark text
        c.drawString(500, 400, watermark_text)  # Position of the watermark text
        c.rotate(-35)

    # Check if image2 exists before adding it
    if image2_path:
        c.drawImage(image2_path, 0, 0, width=image2.width, height=image2.height)

    c.restoreState()
    c.save()

pdf_merger = PdfMerger()
for i, url in enumerate(urls, start=1):
    output_pdf = f"output_{i}.pdf"
    pdf_merger.append(output_pdf)

final_output_pdf = "final_output.pdf"
pdf_merger.write(final_output_pdf)
pdf_merger.close()
print("Done")
# print link to the final output file so I can open it from the console
print("Final output file:", os.path.abspath(final_output_pdf))

if you want to convert the pdf into a html run below snippet

In [ ]:
import fitz,base64  

input_pdf_path = "final_output.pdf"  # Replace with the path to your input PDF file
output_html_path = "output.html"  # Replace with the desired output HTML file path
pdf_document = fitz.open(input_pdf_path)
    
    # Create an empty HTML document
html_document = ""

# Loop through PDF pages
for page_num in range(len(pdf_document)):
    page = pdf_document[page_num]
    pix = page.get_pixmap()

    # Save the pixmap as an image
    image_path = f"page_{page_num}.png"
    pix.save(image_path, "png")

    # Read the image as bytes and encode it as base64
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()
        base64_image = base64.b64encode(image_data).decode("utf-8")

    # Embed the base64 image in the HTML document
    img_tag = f'<img src="data:image/png;base64,{base64_image}" /><br>'
    html_document += img_tag

# Save the HTML document to a file
with open(output_html_path, "w", encoding="utf-8") as html_file:
    html_file.write(html_document)